In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb

from matplotlib import pyplot as plt

import distfit
import scipy

sb.set_theme(palette="rocket")

In [ ]:
# distfit stuff

def plot_distfit_result(
    fitter: distfit.distfit,
    title: str = "",
    num_top: int = 3,
    line_colour: str = "#BBBBBB",
    pdf_linewidth: int = 2,
    pdf_bar_multiplier: float = 0.5,

) -> plt.Figure:
    '''
    pdf_color accepts hex strings only. (0xFFFFFF)
    '''

    bar_props = {
        "color": line_colour
    }

    pdf_props = {
        "color": manip_rgb(line_colour, pdf_bar_multiplier),
        "linewidth": pdf_linewidth,
    }

    fig, ax = fitter.plot(
        chart="pdf",
        title=title,
        n_top=num_top,
        pdf_properties=pdf_props,
        bar_properties=bar_props,
        figsize=(12, 8),
    )

    return fig


def manip_rgb(input: str, amount: float) -> str:
    '''Input is a hash-prefixed RGB string: "#FFFFFF"'''

    input_filt = input[1:].ljust(6, "0")

    r = int(float(int(input_filt[0:2], 16)) * amount) % 255
    g = int(float(int(input_filt[2:4], 16)) * amount) % 255
    b = int(float(int(input_filt[4:6], 16)) * amount) % 255

    r_str = hex(r)[2:].ljust(2, "0")
    g_str = hex(g)[2:].ljust(2, "0")
    b_str = hex(b)[2:].ljust(2, "0")

    res = f"#{r_str}{g_str}{b_str}"

    return res

# Determining warmup time
As a general rule of thumb, we want to start measuring when all base stations are working at maximum capacity.

In [ ]:
# single run of 0 channel reserve
ev_r0 = pd.read_csv("simulator_events_a_r0_r1_e10000.csv")
# single run of 1 channel reserve
ev_r1 = pd.read_csv("simulator_events_a_r1_r1_e10000.csv")

ev_r0["station"].value_counts().sort_index()
ev_r0.info()

In [ ]:
expl = ev_r0[ev_r0["station"] <= 2].explode(column="station")

# sb.lineplot(data=expl, x="time", y="station_free_channels", hue="station", err_style="band", linewidth=0.2)

hist_data = ev_r0[["station", "station_free_channels"]]
hist_data = hist_data.astype({"station": "category"})
new_hist = hist_data.explode("station")

new_hist.info()
fig = plt.figure(figsize=(16, 8))

sb.histplot(data=new_hist, x="station_free_channels", hue="station", multiple="dodge", bins=10, palette="icefire", legend=True)
# plt.legend(loc="best")

# fig.legend()
# move the legend to the right
plt.show()
# hist_data.info()

In [ ]:
ev_r0.info()
# partition the dataframe by station
ev_r0_stations = [ev_r0[ev_r0["station"] == i] for i in range(1, 20)]
ev_r1_stations = [ev_r1[ev_r1["station"] == i] for i in range(1, 20)]

# ev_r0_piv = ev_r0.pivot(index="time", columns="station", values="station_free_channels")
ev_r0_piv = ev_r0.pivot(columns="station", index="time", values="station_free_channels")# index="time", values="station_free_channels")

# ev_r0_piv = ev_r0.melt()

# fill nans with the most recent value
ev_r0_piv.fillna(method="ffill", inplace=True)
ev_r0_piv.fillna(value=10, inplace=True)


# sb.scatterplot(ev_r0_stations[0], x="time", y="station_free_channels")

# create a rolling average of the number of free channels
# for i in range(4):
    # ev_r0_stations[i]["station_free_channels"] = ev_r0_stations[i]["station_free_channels"].rolling(100).mean()
    # ev_r1_stations[i]["station_free_channels"] = ev_r1_stations[i]["station_free_channels"].rolling(100).mean()

fig, ax = plt.figure(figsize=(16,8)), plt.axes()

for station in ev_r0_stations[10:11]:
    ax.axhline(y=station["station_free_channels"].mean(), color='r', linestyle='--', alpha=0.9)

    rolling_small = station["station_free_channels"].rolling(10).mean()
    rolling_large = station["station_free_channels"].expanding().mean()

    sb.scatterplot(data=station, x="time", y=rolling_small, ax=ax, alpha=0.4, marker=".")
    sb.lineplot(data=station, x="time", y=rolling_large, ax=ax, alpha=0.5)

plt.title("single channnel availability over time, 0 channel reserve", fontsize=15)
plt.savefig("channel_availability_r0.svg")
plt.show()
#
# generate a smoothed line plot of the number of free channels over time
# fig, ax = plt.subplots(2, 2, figsize=(12, 8))
# for i in range(4):

#     rolling_small = ev_r0_stations[i]["station_free_channels"].rolling(10).mean()
#     rolling_large = ev_r0_stations[i]["station_free_channels"].rolling(100).mean()

#     #plot the mean as a horizontal line
#     ax[i // 2, i % 2].axhline(y=ev_r0_stations[i]["station_free_channels"].mean(), color='b', linestyle='--')

#     sb.lineplot(data=ev_r0_stations[i], x="time", y=rolling_small, ax=ax[i // 2, i % 2], alpha=0.5)
#     sb.lineplot(data=ev_r0_stations[i], x="time", y=rolling_large, ax=ax[i // 2, i % 2], alpha=0.8)
#     ax[i // 2, i % 2].set_title(f"Station {i + 1} - 0 Channel Reserve")
#     ax[i // 2, i % 2].set_xlabel("Time (s)")
#     ax[i // 2, i % 2].set_ylabel("Free Channels")

# plt.tight_layout()
# plt.savefig("station_free_channels_r0.png")
# plt.show()


In [ ]:
perf_anti_r0 = pd.read_csv("simulator_perf_a_r0_r100000_e10000.csv")
perf_anti_r0.info()

perf_anti_r1 = pd.read_csv("simulator_perf_a_r1_r100000_e10000.csv")

In [ ]:
# sb.histplot(perf_df, x="blocked_calls")
# sb.histplot(perf_df, x="dropped_calls")

num_bins = 50

plt.figure(figsize=(12, 9))
sb.histplot(perf_anti_r0, bins=num_bins, kde=True, palette="icefire") # bins=1000)
plt.title("0 channel reservation", fontsize=20)
plt.savefig("reserve-0.svg")
plt.show()

plt.figure(figsize=(12, 9))
sb.histplot(perf_anti_r1, bins=num_bins, kde=True, palette="icefire") # bins=1000)
plt.title("1 channel reservation", fontsize=20)
plt.savefig("reserve-1.svg")
plt.show()

In [ ]:
no_res_blocked = perf_anti_r0["blocked_calls"]

fitter = distfit.distfit(distr="norm", smooth=6)
fit_results = fitter.fit_transform(no_res_blocked, verbose=False)
f = plot_distfit_result(fitter, "blocked calls percentage - 0 channel reserve")
f.savefig("reserve-0_blocked.svg")
fit_results["model"]

In [ ]:
no_res_blocked = perf_anti_r0["dropped_calls"]

fitter = distfit.distfit(distr="norm", smooth=6)
fit_results = fitter.fit_transform(no_res_blocked, verbose=False)
f = plot_distfit_result(fitter, "dropped calls percentage - 0 channel reserve")
f.savefig("reserve-0_dropped.svg")
fit_results["model"]

In [ ]:
res_blocked = perf_anti_r1["blocked_calls"]

fitter = distfit.distfit(distr="norm", smooth=6)
fit_results = fitter.fit_transform(res_blocked, verbose=False)
f = plot_distfit_result(fitter, "blocked calls percentage - 1 channel reserve")
f.savefig("reserve-1_blocked.svg")
fit_results["model"]

In [ ]:
res_blocked = perf_anti_r1["dropped_calls"]

fitter = distfit.distfit(distr="norm", smooth=4)
fit_results = fitter.fit_transform(res_blocked, verbose=False)
f = plot_distfit_result(fitter, "dropped calls percentage - 1 channel reserve")
f.savefig("reserve-1_dropped.svg")
fit_results["model"]

# Paired t-test

In [ ]:
import scipy.stats as stats


x = perf_anti_r0["blocked_calls"]
y = perf_anti_r1["blocked_calls"]

print(stats.ttest_rel(x, y))

a = perf_anti_r0["dropped_calls"]
b = perf_anti_r1["dropped_calls"]


print(stats.ttest_rel(a, b))